In [2]:
"""
A4 -> Booklet (A5/A6/A7) prototype desktop app
Built with: PyQt6 (GUI) and PyMuPDF (fitz) for PDF handling

This file is a working prototype focused on correctness and clarity.
It:
 - accepts an A4 PDF
 - chooses a fold target (A5/A6/A7)
 - performs simple signature planning using 28/32 balancing logic (as described by the user)
 - arranges pages into sheets (imposition) and creates a print-ready PDF

Notes / Limitations (prototype):
 - The imposition algorithm is generic and works by pulling page numbers from the
   outside-in to build each sheet. This pattern is commonly used for booklet imposition.
 - Generated output places rasterized page content (from original pages) scaled and
   positioned into new A4 pages representing the printed sheet (both sides handled
   as separate pages in the output PDF; user/printer must print double-sided and
   flip on the long edge).
 - You should thoroughly test with sample PDFs and with your printer workflow
   (front/back flipping conventions vary by printer).

Requirements:
 pip install PyQt6 PyMuPDF

Run:
 python A4_to_Booklet_PyQt6_PyMuPDF.py

"""

import sys
import math
import fitz  # PyMuPDF
from PyQt6 import QtWidgets, QtCore
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QFileDialog, QLabel, QPushButton, QComboBox, QVBoxLayout, QWidget, QTextEdit
)

# ------------------------- Utility functions -------------------------

def panels_for_target(target: str):
    """Return panels per sheet for each fold target and panels per side."""
    if target == "A5":
        return 4, 2
    if target == "A6":
        return 8, 4
    if target == "A7":
        return 16, 8
    raise ValueError("Unknown target")


def plan_signatures(total_pages: int, small_sig=28, large_sig=32):
    """
    Signature planning algorithm matching the description in the prompt.
    Idea: try to pack pages with mostly small signatures (e.g. 28), then adjust
    some groups to large_sig to consume the remainder evenly (using difference multiples of 4 logic).

    Returns list of signature sizes summing >= total_pages (padding will be added later)
    """
    if total_pages <= small_sig:
        return [small_sig]

    base_count = total_pages // small_sig  # how many small sigs fully
    remainder = total_pages - base_count * small_sig

    # If remainder is zero, done. Otherwise we try converting some smalls into larges
    if remainder == 0:
        return [small_sig] * base_count

    # We need to find k such that remainder + k*small_sig can be expressed as m*large_sig + leftover
    # Simpler: follow user logic: find how many small signatures to convert to large by grouping
    # Compute how many "extra pages" we need: extra = remainder
    extra = remainder

    # We will convert some small to large. Each conversion adds (large_sig - small_sig) pages.
    diff = large_sig - small_sig  # typically 4
    # how many conversions required to absorb the remainder into multiples of small_sig?
    # Use an algorithm from prompt: find t so that extra = t * diff + r where r will be padded
    conversions = 0
    while conversions <= base_count and extra > 0:
        if extra % diff == 0:
            break
        conversions += 1
        extra += small_sig  # we effectively move one small into the "tail" (see prompt logic)

    # fallback: greedily convert as many as needed to keep signature counts integral
    # Simpler realistic approach: greedily convert while remainder > 0
    conversions = 0
    rem = remainder
    while rem > 0 and conversions < base_count:
        conversions += 1
        rem = remainder + conversions * diff
        if rem % small_sig == 0:
            break

    # Compose signature list
    sigs = []
    # keep (base_count - conversions) small_sig
    keep_small = max(base_count - conversions, 0)
    sigs.extend([small_sig] * keep_small)
    # add conversions many large_sig
    sigs.extend([large_sig] * conversions)

    # now compute leftover pages after those
    used = sum(sigs)
    leftover = total_pages - used
    if leftover > 0:
        # if still leftover, add one more signature of nearest size (prefer large)
        if leftover <= small_sig:
            sigs.append(small_sig)
        else:
            sigs.append(large_sig)

    # final safety: if total pages still larger than sum, add another small
    while sum(sigs) < total_pages:
        sigs.append(small_sig)

    return sigs


def paginate_signature(sig_size: int, panels_per_sheet: int):
    """
    Create an imposition layout for a signature of size sig_size.
    Returns a list of sheets. Each sheet is a dict with 'front' and 'back', each a list
    of page indices (1-based), with -1 indicating a blank (padding) page.

    Strategy: build a deque-like pages list and for each sheet take pairs (last, first) repeatedly
    to form one sheet group of length panels_per_sheet.
    """
    from collections import deque
    pages = deque(range(1, sig_size + 1))
    sheets = []
    half = panels_per_sheet // 2

    while pages:
        group = []
        for k in range(half):
            # take last
            if pages:
                group.append(pages.pop())
            else:
                group.append(-1)
            # take first
            if pages:
                group.append(pages.popleft())
            else:
                group.append(-1)
        # group length == panels_per_sheet
        # front side: first half, back side: last half but with mirrored order for correct folding
        front = group[:half]
        back = list(reversed(group[half:]))
        sheets.append({"front": front, "back": back})
    return sheets


def map_signature_to_global_pages(start_page: int, sig_size: int, sheets):
    """
    Convert local signature-relative page numbers to global page numbers (absolute across whole document).
    start_page is the 1-based global index of the first page of this signature in the original doc.
    """
    mapped = []
    for sheet in sheets:
        front = [ (start_page + (p - 1)) if p > 0 else -1 for p in sheet['front'] ]
        back = [ (start_page + (p - 1)) if p > 0 else -1 for p in sheet['back'] ]
        mapped.append({"front": front, "back": back})
    return mapped


def render_sheet_to_pdf(src_doc: fitz.Document, page_numbers_front, page_numbers_back, a4_width=595, a4_height=842):
    """
    Produce two A4 pages in a new PDF representing the front and back of one printed sheet.
    page_numbers_front / back: lists of global page indices or -1 for blank.

    For the front A4 page we place panels_per_side items across the page in a row/column layout.
    We'll arrange panels vertically in a single column pairs (for foldings) by dividing the A4 area recursively.

    Returns fitz.Document() containing two pages (front then back).
    """
    new = fitz.open()

    # Prepare rectangles for panels by recursively splitting an A4 rect into panels_per_sheet regions.
    panels_per_side = len(page_numbers_front)
    panels_per_sheet = panels_per_side * 2

    # We will create panels arranged in a single column of panels_per_sheet//2 rows per side, two columns across (left/right)
    # A safer general approach: split A4 into panels_per_sheet panels using a simple grid:
    cols = 1 if panels_per_side == 1 else 2  # two columns if more than 1 panel per side
    rows = panels_per_side // cols

    # Create front page
    front_page = new.new_page(width=a4_width, height=a4_height)
    # draw each panel
    idx = 0
    for c in range(cols):
        for r in range(rows):
            if idx >= panels_per_side:
                break
            # compute rectangle for this panel
            panel_w = a4_width / cols
            panel_h = a4_height / rows
            x0 = c * panel_w
            y0 = r * panel_h
            rect = fitz.Rect(x0, y0, x0 + panel_w, y0 + panel_h)
            pnum = page_numbers_front[idx]
            if pnum != -1:
                src = src_doc.load_page(pnum - 1)
                # get pixmap of source page and place it scaled to rect
                mat = fitz.Matrix(min(panel_w / src.rect.width, panel_h / src.rect.height),
                                  min(panel_w / src.rect.width, panel_h / src.rect.height))
                pix = src.get_pixmap(matrix=mat, alpha=False)
                img_rect = fitz.Rect(rect.x0, rect.y0, rect.x0 + pix.width, rect.y0 + pix.height)
                front_page.insert_image(img_rect, pixmap=pix)
            idx += 1

    # Create back page
    back_page = new.new_page(width=a4_width, height=a4_height)
    idx = 0
    for c in range(cols):
        for r in range(rows):
            if idx >= panels_per_side:
                break
            panel_w = a4_width / cols
            panel_h = a4_height / rows
            x0 = c * panel_w
            y0 = r * panel_h
            rect = fitz.Rect(x0, y0, x0 + panel_w, y0 + panel_h)
            pnum = page_numbers_back[idx]
            if pnum != -1:
                src = src_doc.load_page(pnum - 1)
                mat = fitz.Matrix(min(panel_w / src.rect.width, panel_h / src.rect.height),
                                  min(panel_w / src.rect.width, panel_h / src.rect.height))
                pix = src.get_pixmap(matrix=mat, alpha=False)
                img_rect = fitz.Rect(rect.x0, rect.y0, rect.x0 + pix.width, rect.y0 + pix.height)
                back_page.insert_image(img_rect, pixmap=pix)
            idx += 1

    return new

# ------------------------- PyQt6 GUI -------------------------

class BookletMakerGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("A4 -> Booklet (A5/A6/A7) - Prototype")
        self.resize(700, 400)

        self.file_path = None
        self.src_doc = None

        # Widgets
        self.label = QLabel("No file selected")
        self.combo = QComboBox()
        self.combo.addItems(["A5", "A6", "A7"])
        self.load_btn = QPushButton("Load A4 PDF")
        self.process_btn = QPushButton("Create Booklet PDF")
        self.log = QTextEdit()
        self.log.setReadOnly(True)

        layout = QVBoxLayout()
        layout.addWidget(self.label)
        layout.addWidget(self.load_btn)
        layout.addWidget(QLabel("Choose fold target:"))
        layout.addWidget(self.combo)
        layout.addWidget(self.process_btn)
        layout.addWidget(QLabel("Log & Notes:"))
        layout.addWidget(self.log)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        # Connections
        self.load_btn.clicked.connect(self.load_pdf)
        self.process_btn.clicked.connect(self.create_booklet)

    def log_msg(self, *args):
        self.log.append(" ".join(str(a) for a in args))

    def load_pdf(self):
        path, _ = QFileDialog.getOpenFileName(self, "Open A4 PDF", filter="PDF Files (*.pdf)")
        if not path:
            return
        try:
            doc = fitz.open(path)
            # basic A4 validation (use first page size; allow slight tolerances)
            p0 = doc.load_page(0)
            w, h = p0.rect.width, p0.rect.height
            # A4 in points: 595 x 842 (approx)
            if not (570 <= w <= 620 and 820 <= h <= 860):
                self.log_msg(f"Warning: first page size looks like {w:.0f}x{h:.0f} points (not strict A4).")
            self.file_path = path
            self.src_doc = doc
            self.label.setText(f"Loaded: {path} — {doc.page_count} pages")
            self.log_msg(f"Loaded PDF with {doc.page_count} pages")
        except Exception as e:
            self.log_msg("Failed to open PDF:", e)

    def create_booklet(self):
        if not self.src_doc:
            self.log_msg("Please load a source PDF first.")
            return
        target = self.combo.currentText()
        panels, panels_per_side = panels_for_target(target)
        total_pages = self.src_doc.page_count

        self.log_msg(f"Target: {target} — {panels} panels per sheet ({panels_per_side} per side)")
        self.log_msg("Planning signatures...")
        sigs = plan_signatures(total_pages)
        self.log_msg("Signatures chosen:", sigs)

        # create output PDF
        out_pdf = fitz.open()

        current_start = 1
        for sig in sigs:
            # ensure sig is multiple of panels (if not, pad to nearest multiple)
            panels_per_sheet = panels
            if sig % panels_per_sheet != 0:
                pad = panels_per_sheet - (sig % panels_per_sheet)
                self.log_msg(f"Padding signature of size {sig} with {pad} blank pages to fit panels")
                sig += pad

            self.log_msg(f"Imposing signature starting at global page {current_start} size {sig}")
            # compute sheets for this signature
            local_sheets = paginate_signature(sig, panels_per_sheet)
            global_sheets = map_signature_to_global_pages(current_start, sig, local_sheets)

            # render each sheet
            for s in global_sheets:
                front = s['front']
                back = s['back']
                # replace pages beyond original total with -1
                def fix_list(lst):
                    return [p if (p != -1 and p <= total_pages) else -1 for p in lst]
                front = fix_list(front)
                back = fix_list(back)
                rendered = render_sheet_to_pdf(self.src_doc, front, back)
                # append pages of rendered to out_pdf
                out_pdf.insert_pdf(rendered)
                rendered.close()

            current_start += sig

        # Save
        save_path, _ = QFileDialog.getSaveFileName(self, "Save Booklet PDF", filter="PDF Files (*.pdf)")
        if not save_path:
            self.log_msg("Save cancelled")
            return
        out_pdf.save(save_path)
        out_pdf.close()
        self.log_msg("Saved booklet to:", save_path)
        QtWidgets.QMessageBox.information(self, "Done", f"Booklet created and saved to:\n{save_path}")


# ------------------------- Run App -------------------------

def main():
    app = QApplication(sys.argv)
    win = BookletMakerGUI()
    win.show()
    sys.exit(app.exec())

if __name__ == '__main__':
    main()


SystemExit: 0